# Neural Network Regression Model

In [11]:
# Importing libraries

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline


In [12]:
# Setting random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

In [13]:
# Loading in the data 
X_train = pd.read_csv('../data/train_set_X.csv').values
y_train = pd.read_csv('../data/train_set_y.csv').values.ravel()


Now that we have loaded in the data, we define our multilayer perceptron model for regression, where we define input dimensions, output dimensions, number of hidden layers, number of hidden units, dropout rate, and activation functions. 

In [14]:
# Defining the MLP model

class MLP(nn.Module): 
    # Initializing the model
    def __init__(self, input_size, hidden_layers, hidden_units, dropout_rate): 
        super(MLP, self).__init__() # Initialize the nn.Module superclass
        layers = [] # List to hold the layers of the network
        input_features = input_size # Number of input features
        
        # Adding hidden layers
        for _ in range(hidden_layers): # Loop to create hidden layers
            layers.append(nn.Linear(input_features, hidden_units)) # Linear layer
            layers.append(nn.ReLU()) # Activation function
            layers.append(nn.Dropout(dropout_rate)) # Dropout layer
            input_features = hidden_units # Update input features for next layer
            
        # Adding output layer
        layers.append(nn.Linear(input_features, 1)) # Output layer
        self.nn = nn.Sequential(*layers) # Sequential container for the layers

    # Forward pass
    def forward(self, x):
        return self.nn(x)

Now that we have our MLP in pytorch, we create a wrapper class so that it is compatible with scikit-learn's GridSearchCV for hyperparameter tuning. In this wrapper class, we define the initialization, fitting, and prediction methods.

In [15]:
# Creating sklearn wrapper for PyTorch model

class NeuralNet(BaseEstimator, RegressorMixin):
    # Initializing the regressor with hyperparameters
    def __init__(self, hidden_layers=2, hidden_units=64, dropout_rate=0.2, 
                 learning_rate=0.001, n_epochs=500):
        self.hidden_layers = hidden_layers # Number of hidden layers
        self.hidden_units = hidden_units # Number of units in each hidden layer
        self.dropout_rate = dropout_rate # Dropout rate
        self.learning_rate = learning_rate # Learning rate for optimizer
        self.n_epochs = n_epochs # Number of training epochs
        self.model = None # Placeholder for the model
        
        # Fitting the model
    def fit(self, X, y):
        # Convert to PyTorch tensors 
        X_tensor = torch.FloatTensor(X)
        y_tensor = torch.FloatTensor(y).reshape(-1, 1)
        
        # Initialize model
        input_size = X.shape[1]
        self.model = MLP(input_size, self.hidden_layers, 
                        self.hidden_units, self.dropout_rate)
        
        # Loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        
        # Training loop
        self.model.train()
        for epoch in range(self.n_epochs):
            predictions = self.model(X_tensor) # Forward pass
            loss = criterion(predictions, y_tensor) # Compute loss
            optimizer.zero_grad() # Zero gradients
            loss.backward() # Backward pass
            optimizer.step() # Update weights
        
        return self
    
    # Making predictions
    def predict(self, X):
        self.model.eval() # Set model to evaluation mode
        with torch.no_grad(): # No gradient computation
            X_tensor = torch.FloatTensor(X) # Convert to tensor
            predictions = self.model(X_tensor) # Forward pass
        return predictions.numpy().flatten() # Return as numpy array

Now that the neural network is defined, we create a pipeline that begins with the standard scaler for feature scaling and then the neural network regressor.

In [16]:
# Creating pipeline with scaler and PyTorch regressor

nn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('nn', NeuralNet())
])

We now create a hyperparameter grid to tune the model using grid search with cross-validation and r2 as the scoring metric.

In [17]:
# Defining hyperparameter grid
# Tuning hidden layers, hidden units, dropout rate, learning rate, and epochs

param_grid = {
    'nn__hidden_layers': [1, 2, 3],
    'nn__hidden_units': [32, 64, 128],
    'nn__dropout_rate': [0.1, 0.2, 0.3],
    'nn__learning_rate': [0.001, 0.01, 0.1],
    'nn__n_epochs': [50, 100, 150]
}

In [18]:
# Performing Grid Search with Cross-Validation (5 folds)

grid_search = GridSearchCV(
    estimator=nn_pipeline,
    param_grid=param_grid,
    cv=5,
    n_jobs = 1,
    scoring='r2',
    verbose=3
)

# Fitting the grid search to the training data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV 1/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=50;, score=-4.595 total time=   0.1s
[CV 2/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=50;, score=-5.642 total time=   0.0s
[CV 3/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=50;, score=-4.809 total time=   0.0s
[CV 4/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=50;, score=-6.015 total time=   0.0s


/Users/stefanregalia/Desktop/ds4021-final-project/.venv/lib/python3.13/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/Users/stefanregalia/Desktop/ds4021-final-project/.venv/lib/python3.13/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/Users/stefanregalia/Desktop/ds4021-final-project/.venv/lib/python3.13/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the

[CV 5/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=50;, score=-4.373 total time=   0.0s
[CV 1/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=100;, score=-4.306 total time=   0.0s
[CV 2/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=100;, score=-5.166 total time=   0.0s
[CV 3/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=100;, score=-4.513 total time=   0.0s
[CV 4/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=100;, score=-5.435 total time=   0.0s
[CV 5/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=100;, score=-4.520 total time=   0.0s


/Users/stefanregalia/Desktop/ds4021-final-project/.venv/lib/python3.13/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/Users/stefanregalia/Desktop/ds4021-final-project/.venv/lib/python3.13/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/Users/stefanregalia/Desktop/ds4021-final-project/.venv/lib/python3.13/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the

[CV 1/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=150;, score=-3.837 total time=   0.1s
[CV 2/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=150;, score=-4.590 total time=   0.0s
[CV 3/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=150;, score=-3.969 total time=   0.1s
[CV 4/5] END nn__dropout_rate=0.1, nn__hidden_layers=1, nn__hidden_units=32, nn__learning_rate=0.001, nn__n_epochs=150;, score=-4.507 total time=   0.1s


...

[CV 3/5] END nn__dropout_rate=0.3, nn__hidden_layers=3, nn__hidden_units=128, nn__learning_rate=0.1, nn__n_epochs=150;, score=-2.216 total time=   0.4s


/Users/stefanregalia/Desktop/ds4021-final-project/.venv/lib/python3.13/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


[CV 4/5] END nn__dropout_rate=0.3, nn__hidden_layers=3, nn__hidden_units=128, nn__learning_rate=0.1, nn__n_epochs=150;, score=-3.165 total time=   0.3s
[CV 5/5] END nn__dropout_rate=0.3, nn__hidden_layers=3, nn__hidden_units=128, nn__learning_rate=0.1, nn__n_epochs=150;, score=-1.174 total time=   0.3s


/Users/stefanregalia/Desktop/ds4021-final-project/.venv/lib/python3.13/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


,estimator,Pipeline(step...NeuralNet())])
,param_grid,"{'nn__dropout_rate': [0.1, 0.2, ...], 'nn__hidden_layers': [1, 2, ...], 'nn__hidden_units': [32, 64, ...], 'nn__learning_rate': [0.001, 0.01, ...], ...}"
,scoring,'r2'
,n_jobs,1
,refit,True
,cv,5
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,copy,True


After grid search with cross validation, we extract the best parameters, best model, and best score to find the optimal neural network regression model for this task.

In [19]:
# Finding the best parameters and model
best_params_nn = grid_search.best_params_
print("Best Parameters:", best_params_nn)

best_model_nn = grid_search.best_estimator_
print("Best model:", best_model_nn)

Best Parameters: {'nn__dropout_rate': 0.3, 'nn__hidden_layers': 3, 'nn__hidden_units': 32, 'nn__learning_rate': 0.1, 'nn__n_epochs': 50}
Best model: Pipeline(steps=[('scaler', StandardScaler()),
                ('nn',
                 NeuralNet(dropout_rate=0.3, hidden_layers=3, hidden_units=32,
                           learning_rate=0.1, n_epochs=50))])


In [20]:
# Printing the best score
grid_search.best_score_

np.float64(-0.05130305691061303)